<img src="logo.png" width="420px">

# `TODOIST AUTOMATION`

### `Settings`

#### Libraries

In [ ]:
import requests
import json
import uuid
from datetime import datetime, timedelta, timezone

#### Dependences

In [ ]:
import credentials as cr

###  `Data` 

#### Estraction

In [ ]:
api_token = cr.api_token #SELECT YOUR TODOIST API TOKEN HERE
headers = {
    "Content-Type": "application/json",
    "X-Request-Id": "un_valor_único_por_cada_solicitud",
    "Authorization": f"Bearer {api_token}"
}

In [ ]:
def get_projects():
    data = {
        "sync_token": "*",
        "resource_types": json.dumps(['projects'])
    }
    response = requests.post("https://api.todoist.com/sync/v9/sync", headers=headers, data=json.dumps(data))
    all_projects = response.json().get('projects', [])
    return all_projects

In [ ]:
def get_tasks():
    data = {
        "sync_token": "*",
        "resource_types": json.dumps(['items'])
    }
    response = requests.post("https://api.todoist.com/sync/v9/sync", headers=headers, data=json.dumps(data))
    all_tasks = response.json()["items"]
    task_dict_id={}
    task_dict_name={}
    for task in all_tasks:
        task_dict_name[task['content']]=[task['id'],projects_dict_id[task['project_id']]]
        task_dict_id[task['id']]=[task['content'],projects_dict_id[task['project_id']]]
    return all_tasks, task_dict_id, task_dict_name

In [ ]:
def get_sections():
    data = {
        "sync_token": "*",
        "resource_types": json.dumps(['sections'])
    }
    response = requests.post("https://api.todoist.com/sync/v9/sync", headers=headers, data=json.dumps(data))
    all_sections = response.json()["sections"]
    return all_sections

#### Projects

In [ ]:
all_projects = get_projects()
projects_dict_id={}
projects_dict_name={}
for project in all_projects:
    projects_dict_id[project['id']]=project['name']
    projects_dict_name[project['name']]=project['id']
projects_dict_id

#### Tasks

In [ ]:
all_tasks,task_dict_id,task_dict_name=get_tasks()

In [ ]:
all_tasks[200]

#### Sections

In [ ]:
all_sections=get_sections()

In [ ]:
sections_dict_id={}
sections_dict_name={}
for section in all_sections:
    sections_dict_id[section['id']]=section['name']
    sections_dict_name[section['name']]=section['id']
sections_dict_id

#### Test

In [ ]:
index=150
print(all_tasks[index]['content'], "-", all_tasks[index]['due']['date'], "-", projects_dict_id[all_tasks[index]['project_id']])

#### Labels

In [ ]:
label_names=[]
for task in all_tasks:
    gross_labels=task['labels']
    for label in gross_labels:
        if label not in label_names:
            label_names.append(label)
label_names

### `Orders`

#### Creating tasks

In [ ]:
def priority_inversal(n):
    if n==1:
        return 4
    if n==3:
        return 2
    if n==2:
        return 3
    if n==4:
        return 1

In [ ]:
def create_task(name,project_id,date="",priority=4,labels=[],section=None,parent=None):
    
    priority=priority_inversal(priority)
    
    for label in labels:
        if label not in label_names:
            print("La etiqueta no existe")
            labels=[]

    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {api_token}',
        'X-Request-Id': str(uuid.uuid4()),
    }

    data = {
        'commands': json.dumps([{
            'type': 'item_add',
            'temp_id': str(uuid.uuid4()),
            'uuid': str(uuid.uuid4()),
            'args': {
                'content': name,
                'project_id': project_id,
                'due': {
                    'date': date
                    },
                'priority': priority,
                'labels': labels,
                'section_id': section,
                'parent_id': parent
            }
        }])
    }

    response = requests.post('https://api.todoist.com/sync/v9/sync', headers=headers, data=json.dumps(data))

    if response.status_code == 200:
        print(f"La tarea {name} fue creada exitosamente.")
    else:
        print(f"Error al crear la tarea: {response.status_code}")

#### F1 Calendar

In [ ]:
for i in range(3):    
    create_task(name = f"* GRAN PREMIO {i}",
                project_id = projects_dict_name['Fórmula 1'],
                priority = 4,
                labels = ['Short'],
                section = sections_dict_name['2025'])
    
all_tasks,task_dict_id,task_dict_name=get_tasks()
for i in range(3):
    for j in range(3):
        create_task(name=f'Practice {j+1}',
                    project_id=projects_dict_name['Fórmula 1'],
                    priority=3,
                    labels=['Med'],
                    section=sections_dict_name['2025'],
                    parent=task_dict_name[f'* GRAN PREMIO {i}'][0])
    create_task(name=f'Qualifying',
            project_id=projects_dict_name['Fórmula 1'],
            priority=1,
            labels=['Med'],
            section=sections_dict_name['2025'],
            parent=task_dict_name[f'* GRAN PREMIO {i}'][0])
    create_task(name=f'Race',
            project_id=projects_dict_name['Fórmula 1'],
            priority=1,
            labels=['Long'],
            section=sections_dict_name['2025'],
            parent=task_dict_name[f'* GRAN PREMIO {i}'][0])


#### Task tracking (it desn't work for completed tasks)

In [ ]:
MINUTES=500
time_delta = datetime.now(timezone.utc) - timedelta(MINUTES)
timestamp = time_delta.timestamp()

recently_completed_tasks = [
    item for item in all_tasks if item.get('checked') == True and
    datetime.strptime(item['date_completed'], "%Y-%m-%dT%H:%M:%SZ").replace(tzinfo=timezone.utc).timestamp() > timestamp
]

# Imprime las tareas completadas recientemente
for task in recently_completed_tasks:
    print(task['content'], task['date_completed'])